### BIBLIOTECAS

In [1]:
import os
import pandas as pd
import numpy as np
import glob

### LEITURA DOS ARQUIVOS

In [2]:
def ler_dados_pickle(caminho_arquivo_pkl):
    """Carrega os dados de um arquivo .pkl em um DataFrame."""
    # Indexar o DataFrame pela coluna 'Data'
    return pd.read_pickle(caminho_arquivo_pkl)

### Nova Chamada 1: Carregar dados de todos os arquivos .pkl no diretório ###
caminho_arquivos_pkl = glob.glob(r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_com_NaNs_preenchidos\*.pkl')

# Definir a pasta de saída desejada
pasta_saida_desejada = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_com_series_temporais_regularizadas'

In [3]:
# # def ler_dados_pickle(caminho_arquivo_pkl):
# #     return pd.read_pickle(caminho_arquivo_pkl)
# # # Caminho completo para o arquivo .pkl
# caminho_arquivos_pkl = r'C:\Users\MarioGoncalves\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_com_NaNs_preenchidos\dados_lameirinho_26_02_2024_sem_missing_values.pkl'

# # Ler o arquivo .pkl em um DataFrame
# df = pd.read_pickle(caminho_arquivos_pkl)

### FUNÇÕES

In [3]:
### FUNÇÕES

## FUNÇÃO 1: Construção do dataframe com uma coluna datetime para melhorar a regularização das séries temporais dos dados
def preprocess_dataframe(df):
    """Realiza o pré-processamento no DataFrame."""
    # Combinar as colunas 'Data' e 'Hora' em uma coluna Datetime
    df['Datetime'] = pd.to_datetime(df['Data'] + df['Hora'], format='%d/%m/%Y%H:%M:%S')
    # Remover as colunas 'Data' e 'Hora'
    df.drop(columns=['Data', 'Hora'], inplace=True)
    # Definir a nova coluna 'Datetime' como índice
    df.set_index('Datetime', inplace=True)
    # Ordenar o DataFrame pelo índice (coluna 'Datetime')
    df.sort_index(inplace=True)
    return df

# ### FUNÇÃO 2: Aplicar a regularização temporal de 1S
# def regularize_dataframe(df):
#     """Regulariza o DataFrame por segundo e interpola os valores ausentes."""
#     # Obter os períodos ausentes formatados
#     missing_periods = get_formated_missing_periods(df, tresh_timedelta_str='1S')
#     # Regularizar o DataFrame por segundo e interpolar os valores ausentes
#     df_regularized = regularize_series(df, freq='1S', exclude_ranges=missing_periods)
#     return df_regularized

### FUNÇÃO 2: Aplicar a regularização temporal de 1S
# def regularize_dataframe(df):
#     """Regulariza o DataFrame por segundo e interpola os valores ausentes."""
#     # Obter os períodos ausentes formatados
#     #missing_periods = get_formated_missing_periods(df, tresh_timedelta_str='1S')
#     # Regularizar o DataFrame por segundo e interpolar os valores ausentes
#     df_regularized = regularize_series(df, freq='1S')
#     return df_regularized

### FUNÇÃO 3: Formatar os Missing_values (nao foi aplicada porque os intrvalos de missing values são pequenos entre todos os dados elétricos)
# def get_formated_missing_periods(in_df, tresh_timedelta_str):
#     """Encontra os períodos ausentes no DataFrame."""
#     df = in_df.copy()
#     # Delta da data anterior
#     df['dif'] = df.index.to_series().diff()
#     # Apenas mantém os períodos maiores que tresh_timedelta_str
#     missing_periods_idxs = df[df['dif'] > pd.Timedelta(tresh_timedelta_str)].index
#     # Obtém as posições inteiras
#     int_missing_periods_idxs = np.where(df.index.isin(missing_periods_idxs))[0]
    
#     missing_periods = []
#     # Loop para obter as datas de início e fim para cada período ausente selecionado
#     for int_idx in int_missing_periods_idxs:
#         missing_periods.append([str(df.index[int_idx-1]), str(df.index[int_idx])])
    
#     return missing_periods

### FUNÇÃO 4: Regularizar as séries temporais
def regularize_series(df, freq, exclude_ranges=None, start_date=None, end_date=None):
    """Regulariza o DataFrame para uma frequência específica."""
    if start_date is None:
        start_date = df.index.min()
    if end_date is None:
        end_date = df.index.max()

    # Cria uma série temporal regular sem valores
    reg_freq = pd.DataFrame(index=pd.date_range(start_date, end_date, freq=freq))
    # Combina ambos para obter um DataFrame com todos os Datetimes
    df_add_reg_freq = pd.merge(df, reg_freq, left_index=True, right_index=True, how='outer')

    # Remove linhas dos períodos a não serem interpolados
    if exclude_ranges is not None:
        try:
            for exclude_range in exclude_ranges:
                df_add_reg_freq = df_add_reg_freq.loc[
                    (df_add_reg_freq.index < exclude_range[0]) |
                    (df_add_reg_freq.index > exclude_range[1])]
        except:
            raise ValueError('exclude_dates must be in format [[start, end], ...]')
    
    # Interpola para obter valores em cada intervalo regular
    df_add_reg_freq.interpolate(method='time', inplace=True)
    # Adiciona de volta o período removido, mas com intervalos regulares com NaNs
    df_add_reg_freq = pd.merge(df_add_reg_freq, reg_freq, left_index=True, right_index=True, how='outer')
    # Remove Datetimes fora do intervalo regular
    df_add_reg_freq = df_add_reg_freq.loc[df_add_reg_freq.index.isin(reg_freq.index)]
    df_add_reg_freq.index.set_names(df.index.name, inplace=True)
    
    return df_add_reg_freq

# def add_date_time_columns(df):
#     """Adiciona colunas de Data e Hora ao DataFrame a partir do índice Datetime."""
#     # Extrair colunas de Data e Hora do índice Datetime
#     df['Data'] = df.index.date
#     df['Hora'] = df.index.time
#     return df

### FUNÇÃO 5: Salvar o DataFrame como um arquivo pickle e, opcionalmente, um arquivo CSV ###
def salvar_arquivos(df, caminho_arquivo_pkl, pasta_saida, save_csv=True):
    """Salva o DataFrame como um arquivo pickle e, opcionalmente, um arquivo CSV."""
    caminho_arquivo_pkl_regularizado = os.path.join(pasta_saida, os.path.basename(caminho_arquivo_pkl).replace('.pkl', '_com_serie_regularizada_1S.pkl'))

    # Salvar o arquivo pickle
    df.to_pickle(caminho_arquivo_pkl_regularizado)

    # Salvar o arquivo CSV, se a opção save_csv for True
    if save_csv:
        caminho_arquivo_csv = os.path.join(pasta_saida, os.path.basename(caminho_arquivo_pkl).replace('.pkl', '_com_serie_regularizada_1S.csv'))
        df.to_csv(caminho_arquivo_csv, index=True)

### CHAMADA DAS FUNÇÕES

In [4]:
# Iterar sobre cada caminho de arquivo
for caminho_arquivo_pkl in caminho_arquivos_pkl:
    # Carregar o DataFrame a partir do arquivo .pkl
    df = ler_dados_pickle(caminho_arquivo_pkl)

    ### CHAMADA da Função 1: Corrigir o Data Hora para Datetime (uma coluna apenas)
    df = preprocess_dataframe(df)

    # ### CHAMADA da Função 2: Regularizar dataframe
    # df = regularize_dataframe(df)

    ### CHAMADA da Função 3: 'opcional'

    ### CHAMADA da Função 4: para regularizar a série temporal em escalas de 1 segundo a partir da data de início
    # Determinar a data de início e de fim com base no nome do arquivo



    if 'dados_lameirinho_P 05_02_2024_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-01-29 09:33:08'

    elif 'dados_lameirinho_5_fev_14_fev_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-02-05 09:07:56'

    elif 'dados_lameirinho_P_19_02_2024_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-02-14 09:10:50'

    elif 'dados_lameirinho_26_02_2024_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-02-19 09:56:17'

    elif 'dados_lameirinho_CP_26_02_04_03_2024_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-02-26 09:09:26'

    elif 'dados_lameirinho_CP_11_03_2024_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-03-04 16:25:10'

    elif 'dados_lameirinho_11_03_2024_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-03-11 19:23:53'

    elif 'dados_lameirinho_CP_25_03_2024_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-03-18 16:10:07'

    elif 'dados_lameirinho_P_25_3_08_04_2024_NEW_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-03-25 15:09:05'

    elif 'dados_lameirinho_8_4_15_04_2024_NEW_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-04-08 09:39:41'

    elif 'dados_lameirinho_15_04_22_04_2024_NEW_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-04-15 14:03:05'

    elif 'dados_lameirinho_22_04_29_04_2024_NEW_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-04-22 14:51:11'

    elif 'dados_lameirinho_CP_29_04_06_05_2024_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-04-22 09:06:06'

    elif 'dados_lameirinho_06_05_13_05_2024_sem_excesso_original_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-05-06 17:30:24'

    elif 'dados_lameirinho_E_CP_13_05_20_05_sem_missing_values' in caminho_arquivo_pkl:
        start_date = '2024-05-13 10:22:16'

       
    # Adicionar a chamada da função `regularize_series` com a data específica
    df = regularize_series(df, '1S', start_date=start_date)

    # ### Adicionar colunas de Data e Hora ao DataFrame
    # df = add_date_time_columns(df)

    # ### CHAMADA da Função 5: salvar os arquivos na pasta de saída desejada
    salvar_arquivos(df, caminho_arquivo_pkl, pasta_saida_desejada, save_csv=True)